In [1]:
import pandas as pd
import math,os,fnmatch
from vehicle_crop import *
from PIL import Image

# Load train/test datasets

In [2]:
df_train = pd.read_csv('data/Dataset for participants V2/train.csv')
df_test = pd.read_csv('data/Dataset for participants V2/test.csv')
df_train.head()

,tag_id,image_id,p1_x,p_1y,p2_x,p2_y,p3_x,p3_y,p4_x,p4_y,...,enclosed_cab,spare_wheel,wrecked,flatbed,ladder,enclosed_box,soft_shell_box,harnessed_to_a_cart,ac_vents,color
0,15036,16490,3988.399902,1227.588745,3986.710693,1213.288208,4002.600098,1211.411255,4004.289307,1225.711792,...,0,0,0,0,0,0,0,0,-1,black
1,31658,16490,3380.804152,33.370945,3329.811654,128.465062,3358.064254,142.935906,3409.056752,44.396350,...,-1,-1,0,-1,0,1,0,0,-1,red
2,26971,17122,3948.334473,1426.797241,3958.876221,1410.750122,4000.665527,1438.202759,3990.123779,1454.249878,...,0,0,0,-1,-1,-1,-1,-1,-1,red
3,19301,12193,1977.798007,1009.010733,1983.435998,992.096760,2034.177918,1009.010733,2027.913483,1027.177593,...,0,0,0,-1,-1,-1,-1,-1,-1,silver/grey
4,35906,12193,3196.620177,1284.240545,3193.174738,1311.804057,3288.957942,1321.451286,3292.403381,1293.198686,...,-1,-1,0,1,0,0,0,0,-1,red


# Prepare vehicle images

In [4]:
def prepare_images(df, in_dir, out_dir):
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
        
    images_fname = os.listdir(in_dir)
    last_image_id = None
    for index, row in df.iterrows():
        tag_id = int(row['tag_id'])
        image_id = int(row['image_id'])
        if (image_id != last_image_id):
            fname = fnmatch.filter(images_fname, '{}.*'.format(image_id))[0]
            aerial_img = Image.open('{}/{}'.format(in_dir, fname))

        polygon = [
            (row['p1_x'],row['p_1y']),
            (row[' p2_x'],row[' p2_y']),
            (row[' p3_x'],row[' p3_y']),
            (row[' p4_x'],row[' p4_y'])
        ]
        vehicle_img = prepare_image(aerial_img, polygon)
        vehicle_img.save('{}/{}.bmp'.format(out_dir,tag_id))
    print('Completed images preparation: {}'.format(out_dir))

def prepare_image(source_img, polygon):
    cropped = crop_vehicle(source_img, polygon, 20)
    return make_square(cropped, 10)
    
def make_square(img, min_size=256, fill_color=(0, 0, 0, 0)):
    x, y = img.size
    size = max(min_size, x, y)
    new_img = Image.new('RGB', (size, size), fill_color)
    new_img.paste(img, (int((size - x) / 2), int((size - y) / 2)))
    return new_img

prepare_images(df_train,'data/Dataset for participants V2/training imagery', 'data/train')
prepare_images(df_test,'data/Dataset for participants V2/test imagery', 'data/test')

Completed images preparation: data/train
Completed images preparation: data/test


# Reduce training multiclass labels to binary

In [5]:
df = pd.read_csv('data/Dataset for participants V2/train.csv')
df2 = pd.DataFrame()
df2['imagefn'] = df.apply(lambda row: '{}.bmp'.format(row['tag_id']),axis=1)
df2['small vehicle'] = df.apply(lambda row: int(row['general_class'] == 'small vehicle'),axis=1)
df2['large vehicle'] = df.apply(lambda row: int(row['general_class'] == 'large vehicle'),axis=1)
df2['minibus'] = df.apply(lambda row: int(row['sub_class'] == 'minibus'),axis=1)
df2['hatchback'] = df.apply(lambda row: int(row['sub_class'] == 'hatchback'),axis=1)
df2['sedan'] = df.apply(lambda row: int(row['sub_class'] == 'sedan'),axis=1)
df2['bus'] = df.apply(lambda row: int(row['sub_class'] == 'bus'),axis=1)
df2['minivan'] = df.apply(lambda row: int(row['sub_class'] == 'minivan'),axis=1)
df2['truck'] = df.apply(lambda row: int(row['sub_class'] == 'truck'),axis=1)
df2['van'] = df.apply(lambda row: int(row['sub_class'] == 'van'),axis=1)
df2['jeep'] = df.apply(lambda row: int(row['sub_class'] == 'jeep'),axis=1)
df2['cement mixer'] = df.apply(lambda row: int(row['sub_class'] == 'cement mixer'),axis=1)
df2['dedicated agricultural vehicle'] = df.apply(lambda row: int(row['sub_class'] == 'dedicated agricultural vehicle'),axis=1)
df2['tanker'] = df.apply(lambda row: int(row['sub_class'] == 'tanker'),axis=1)
df2['crane truck'] = df.apply(lambda row: int(row['sub_class'] == 'crane truck'),axis=1)
df2['pickup'] = df.apply(lambda row: int(row['sub_class'] == 'pickup'),axis=1)
df2['light truck'] = df.apply(lambda row: int(row['sub_class'] == 'light truck'),axis=1)
df2['prime mover'] = df.apply(lambda row: int(row['sub_class'] == 'prime mover'),axis=1)
df2['red'] = df.apply(lambda row: int(row['color'] == 'red'),axis=1)
df2['black'] = df.apply(lambda row: int(row['color'] == 'black'),axis=1)
df2['blue'] = df.apply(lambda row: int(row['color'] == 'blue'),axis=1)
df2['silver/grey'] = df.apply(lambda row: int(row['color'] == 'silver/grey'),axis=1)
df2['white'] = df.apply(lambda row: int(row['color'] == 'white'),axis=1)
df2['other'] = df.apply(lambda row: int(row['color'] == 'other'),axis=1)
df2['yellow'] = df.apply(lambda row: int(row['color'] == 'yellow'),axis=1)
df2['green'] = df.apply(lambda row: int(row['color'] == 'green'),axis=1)
df2['red'] = df.apply(lambda row: int(row['color'] == 'red'),axis=1)
df2['sunroof'] = df.apply(lambda row: int(row['sunroof'] == 1),axis=1)
df2['luggage_carrier'] = df.apply(lambda row: int(row['luggage_carrier'] == 1),axis=1)
df2['open_cargo_area'] = df.apply(lambda row: int(row['open_cargo_area'] == 1),axis=1)
df2['enclosed_cab'] = df.apply(lambda row: int(row['enclosed_cab'] == 1),axis=1)
df2['spare_wheel'] = df.apply(lambda row: int(row['spare_wheel'] == 1),axis=1)
df2['wrecked'] = df.apply(lambda row: int(row['wrecked'] == 1),axis=1)
df2['flatbed'] = df.apply(lambda row: int(row['flatbed'] == 1),axis=1)
df2['ladder'] = df.apply(lambda row: int(row['ladder'] == 1),axis=1)
df2['enclosed_box'] = df.apply(lambda row: int(row['enclosed_box'] == 1),axis=1)
df2['soft_shell_box'] = df.apply(lambda row: int(row['soft_shell_box'] == 1),axis=1)
df2['harnessed_to_a_cart'] = df.apply(lambda row: int(row['harnessed_to_a_cart'] == 1),axis=1)
df2['ac_vents'] = df.apply(lambda row: int(row['ac_vents'] == 1),axis=1)
df2.to_csv('data/Dataset for participants V2/train_binarified.csv',index=False)
df2.head(5)

,imagefn,small vehicle,large vehicle,minibus,hatchback,sedan,bus,minivan,truck,van,...,open_cargo_area,enclosed_cab,spare_wheel,wrecked,flatbed,ladder,enclosed_box,soft_shell_box,harnessed_to_a_cart,ac_vents
0,15036.bmp,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,31658.bmp,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
2,26971.bmp,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,19301.bmp,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,35906.bmp,0,1,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
